### Introduction
Gradient Boosting Machine or GBM for short, belongs to the boosting family, unlike Adaptive Boosting ![AdaBoost](./ada_boost.ipynb) which learns to correct the mistakes by putting more weight on difficult to classify instances and find the optimal contribution $\alpha$ for the learner at that iteration, the GBM try to find the residuals at each iteration. 

### Algorithm
In regression analysis, the general form of regression model is expressed as:
$y = \hat{y} + \epsilon$, where $\hat{y}$ is the predicted value of $y$ based on the regression model, and $\epsilon$ is the error term which represent the difference between predicted value $\hat{y}$ and true value $y$. The GBM learn the $\epsilon$ after each time step and expect after each time step the $\epsilon$ will be smaller and smaller, the sum of $\epsilon$ over dataset is defined:
$$L = \sum_{i=1}^{n} \epsilon_i^2 = \sum_{i=1}^{n} (y_i - \hat{y})^2_i$$

The loss function $L$ reaches optimal when $\hat{y}_t = \hat{y} + \epsilon$, and therefore, we define the base learner as $\hat{y}$, and generalize the $\epsilon$ into a function $h(x)$, we got:
$$\hat{y}_t = \hat{y} + h(x) = f(x) + h(x)$$

Where $f(x)$ is considered as base weak learner, and $h(x)$ is weak leaner that is trained to correct the errors (residuals), the algorithm can be generalized as follows:

1. Init base learner: $f(x)$
2. Compute residuals: $\epsilon = y - (f(x) + \sum_{i=1}^{n} h_i(x))$
3. Train weak learner to learn the residuals: h_i(x)
4. Add the weak learner to the set
5. Back to step 2 until we reach the expected residuals sum of squared, or reach the maximum number of estimators


### Implementation
In this implementation, we use the `DecisionTreeClassifer` as base weak leaner and `DecisionTreeRegressor` to estimate the residuals, the dataset in use is `hastie_10_2` from `sklearn.datasets`

In [21]:
import numpy as np
import tqdm
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_hastie_10_2

In [2]:
X, y = make_hastie_10_2()
y[y < 0] = 0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [3]:
def encode_one_hot(y):
    categories, inverse = np.unique(y, return_inverse=True)
    one_hot = np.zeros((y.shape[0], categories.size))
    one_hot[np.arange(y.shape[0]), inverse] = 1
    return one_hot

In [18]:
def predict(base, trees, X, lr = 0.1):
    p = base.predict_proba(X)
    for fn in trees:
        p += lr * fn.predict(X)
    return p

def boost(data, residuals, max_depth=5):
    fn = DecisionTreeRegressor(max_depth=max_depth, splitter="random")
    fn = fn.fit(data, residuals)
    return fn

#### Build base learner to learn check the performance of the first tree

In [15]:
max_depth = 5
base = DecisionTreeClassifier(max_depth=max_depth)
base = base.fit(X_train, y_train)

In [17]:
pred = base.predict_proba(X_test)
sum(np.argmax(pred, axis=1) == y_test) / len(y_test)

0.6505555555555556

In [19]:
one_hot = encode_one_hot(y_train)
trees = []
for i in tqdm.tqdm(range(100)):
    residuals = one_hot - predict(base, trees, X_train)
    fn = boost(X_train, residuals, max_depth)
    trees.append(fn)

100%|██████████| 100/100 [00:02<00:00, 45.41it/s]


In [20]:
pred = predict(base, trees, X_test)
sum(np.argmax(pred, axis=1) == y_test) / len(y_test)

0.9063888888888889

#### Performance before and after boosted
The performance of the base learner is 65% accuracy and after boosted is 90% which is around 38% uplifted in term of accuracy